In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pygam import LinearGAM, s, f, te, l
from sklearn.metrics import mean_squared_error
import math

In [2]:
BASEPATH = 'C:\SuryaMain\Python Projects\Chinook-Growth-Project'
SIMULATION_BASEPATH = os.path.join(BASEPATH, "simulate_nonlinear_data")
datasets = [name for name in os.listdir(SIMULATION_BASEPATH) if name[0:4] == 'data'] 
    
def get_data(fname):
    path = os.path.join(SIMULATION_BASEPATH, fname)
    if os.path.exists(path):
        X = pd.read_csv(os.path.join(path, 'X.csv'))
        X.drop(X.columns[0], axis=1, inplace=True)
        y = pd.read_csv(os.path.join(path, 'y.csv'))
        y.drop(y.columns[0], axis=1, inplace=True)
        return [X, y]
    else: 
        print("Dataset does not exist")
        return [0,0]     

class SimGAM(LinearGAM):
    def __init__(self, X, y, **kwargs):
        super().__init__(**kwargs)
        self.X = X
        self.y = y
        self.trained = False
        
    def get_X(self):
        return self.X
    
    def get_Y(self):
        return self.y
    
    def fill_vals(self, num_vars, var_idx):
        sample = np.zeros(shape=(100, num_vars))
        rand_vals = sorted(np.random.normal(0,1,100))
        for row_idx, row in enumerate(sample):
            row[var_idx] = rand_vals[row_idx]
        return sample    

    def fit_gam(self):
        lams = np.logspace(-5,5,20)*self.X.shape[1]
        super().fit(self.X, self.y)
        super().gridsearch(self.X, self.y, lam=lams)
        self.trained = True
        
    def partial_dependences(self):
        pds = {}
        num_vars = X.shape[1]
        for var in range(num_vars):  # Number of variables
            sample = fill_vals(num_vars, var)
            pdep = super().partial_dependence(term=var, X=sample)
            pds[f'var{var}'] = pdep
        return pds
    
    def calc_aic(self):
        if self.trained == True:
            num_params = len(super().coef_) + 1
            aic = -2*super().loglikelihood(self.X,self.y) + 2*num_params
            return aic
        else: 
            print("Model Has Not Been Trained")

In [8]:
dataset_1 = get_data('data_set_2')
X = dataset_1[0]
y = dataset_1[1]
gam = SimGAMTest(X, y, n_splines=15)
gam.fit_gam()


100% (20 of 20) |########################| Elapsed Time: 0:00:05 Time:  0:00:05


In [20]:
pdeps = gam.partial_dependences()
pd_variances = [np.var(pdeps[f'var{i}']) for i in range(0, gam.get_X().shape[1])]
print(f'Variable: {np.argmax(pd_variances)+1}')
for i in pd_variances:
    print(i)

Variable: 10
0.06948642398944217
0.07117646273905232
0.023813893949529895
0.033929867144076635
0.017013690062256486
0.027136947015530013
0.05891600801773428
0.059940562439568516
0.020094670204767538
0.17558331837031094


In [17]:
def pdp_test(model, nrows, ncols):
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols)
    titles = list(map(lambda x:f'V{x}', np.arange(1,len(X.columns)+1)))
    num_vars = model.X.shape[1]
    for i, ax in enumerate(axs):
        sample = fill_vals(num_vars, i)
        ax.plot(sample[:,i], model.model.partial_dependence(term=i, X=sample))
        ax.plot(sample[:,i], model.model.partial_dependence(term=i, X=sample, width=.95)[1], c='r', ls='--')
        ax.set_title(titles[i]);
        ax.grid(True)
        print(sample.shape)
    plt.suptitle('Partial Dependence', size=16)
    fig.tight_layout()
    plt.show()
    
def pdp(model, nrows, ncols):
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols)
    titles = list(map(lambda x:f'V{x}', np.arange(1,len(X.columns)+1)))
    rand_data = []
    for i, ax in enumerate(axs):
        XX = model.model.generate_X_grid(term=i) 
        #sample = np.random.normal(0,1,100)
        ax.plot(XX[:, i], model.model.partial_dependence(term=i, X=XX))
        ax.plot(XX[:, i], model.model.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
        ax.set_title(titles[i]);
        ax.grid(True)
        rand_data.append(XX)
    plt.suptitle('Partial Dependence', size=16)
    fig.tight_layout()
    plt.show()
    return rand_data


In [ ]:
def main():
    aic_dict = {}
    for dataset in datasets[0:10]:
        dst = get_data(dataset)
        X = dst[0]
        y = dst[1]
        gam = SimGAM(X,y)
        gam.train_lgam(n_splines=15)
        aic = gam.calc_aic()
        aic_dict[dataset] = aic
    return aic_dict

In [ ]:
'''
class SimGAM:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.model = None
        
    def get_X(self):
        return self.X
    
    def get_Y(self):
        return self.y
        
    def train_lgam(self, **kwargs):
        gam = LinearGAM(**kwargs).fit(self.X, self.y)
        lams = []
        if 'lam' in kwargs.keys():
            lams = kwargs.get('lam')
        else:
            lams = np.logspace(-5,5,20)*len(self.X.columns)
        gam.gridsearch(self.X, self.y, lam=lams)
        self.model = gam
        
    def partial_dependences(self):
        pds = {}
        num_vars = X.shape[1]
        for var in range(num_vars):  # Number of variables
            sample = fill_vals(num_vars, var)
            pdep = self.model.partial_dependence(term=var, X=sample)
            pds[f'var{var}'] = pdep
        return pds
    
    def calc_aic(self):
        if self.model is not None:
            num_params = len(self.model.coef_) + 1
            aic = -2*self.model.loglikelihood(self.X,self.y) + 2*num_params
            return aic
        else: 
            print("Model Has Not Been Trained")
            return 0.0
'''